In [1]:
import torch
import torch.functional as F
import torch.nn as nn 

In [2]:
#first commit
# num matrices * batchsize * num inputs
X = torch.rand(5, 2, 3)
X

tensor([[[0.5280, 0.6960, 0.6487],
         [0.2020, 0.3919, 0.3875]],

        [[0.8453, 0.1299, 0.2717],
         [0.9909, 0.7876, 0.3902]],

        [[0.3232, 0.0219, 0.3882],
         [0.0493, 0.6217, 0.7488]],

        [[0.8587, 0.2774, 0.2679],
         [0.2423, 0.6294, 0.3516]],

        [[0.1095, 0.9288, 0.3985],
         [0.5627, 0.7942, 0.0010]]])

In [14]:
# Create the Deep Learning Model
def createTheNet(printtoggle, input_size, num_hidden, num_layers, actfun, bias, batchsize):
    class rnnClassNet(nn.Module):
        def __init__(self, input_size, num_hidden, num_layers, actfun, bias, batchsize):
            
            super().__init__()
            self.printT = printtoggle
            
            # Store Parameters
            self.input_size = input_size
            self.num_hidden = num_hidden
            self.num_layers = num_layers
            self.actfun = actfun
            self.bias = bias
            self.batchsize = batchsize
            
            # # # ----- RNN layers ------ # # #
            # nn.RNN(input_size, num_hidden)
            self.rnn = nn.RNN(input_size, num_hidden,  num_layers,
                              nonlinearity= self.actfun, bias=self.bias)
            
            # # # ----- linear layer ------ # # #
            self.out = nn.Linear(num_hidden, 1)
            
        def forward(self,x):
            
            # initialize hidden state for first input
            hidden = torch.zeros(self.num_layers, self.batchsize, self.num_hidden)
            if self.printT: print(f'Hidden: {list(hidden.shape)}')
            
            # run through the RNN layer
            y, hidden = self.rnn(x, hidden)
            if self.printT: print(f'RNN-out: {list(y.shape)}')
            if self.printT: print(f'RNN-Hidden: {list(hidden.shape)}')
            
            # pass the rnn output through linear layer
            o = self.out(y)
            if self.printT: print(f'Output: {list(o.shape)}')
            
            return o, hidden
    
    modelInstance = rnnClassNet(input_size, num_hidden, num_layers, actfun, bias, batchsize)

    lossfun = nn.BCEWithLogitsLoss()
    
    optimizer = torch.optim.Adam(modelInstance.parameters(), lr=0.001)
    
    return modelInstance, lossfun, optimizer

In [19]:
# network parameters
input_size = 9  # channels/features of data
num_hidden = 16 # breadth of model (number of hidden layers)
num_layers = 1  # depth of model   (number of stacks of hidden layers)
seqlength  = 5  # num of data points used for learning in each segment 
batchsize  = 4  # number of batches of selength
actfun = 'tanh'
bias = True

In [20]:
#Test with 1 batch
net2, lossfun2, optimizer2 = createTheNet(True, input_size, num_hidden, num_layers, actfun, bias, batchsize)
print(net2)

#check all learnable parameters
for p in net2.named_parameters():
    print(f'{p[0]} has size {list(p[1].shape)}')

rnnClassNet(
  (rnn): RNN(9, 16)
  (out): Linear(in_features=16, out_features=1, bias=True)
)
rnn.weight_ih_l0 has size [16, 9]
rnn.weight_hh_l0 has size [16, 16]
rnn.bias_ih_l0 has size [16]
rnn.bias_hh_l0 has size [16]
out.weight has size [1, 16]
out.bias has size [1]


In [21]:
# test the model with some data
X = torch.rand(seqlength, batchsize, input_size)
y = torch.rand(seqlength, batchsize, 1)
yHat, h = net2(X)

# try a loss function
lossfun = nn.MSELoss()
lossfun(yHat, y)


Hidden: [1, 4, 16]
RNN-out: [5, 4, 16]
RNN-Hidden: [1, 4, 16]
Output: [5, 4, 1]


tensor(0.3350, grad_fn=<MseLossBackward0>)